In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
import string
import re
import spacy
import gensim
import warnings
warnings.filterwarnings("ignore")
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import pickle

**Conclusion**

- Topic 0 is related to USA and politics
- Topic 1 is related to  sports.
- Topic 2 is related to AI and healthcare.
- Topic 3 is related to disaster/disease.
- Topic 4 is related to Nigeria and politics.

In [2]:
# loading transformer
sentence_model = pickle.load(open('../sentence_transformer/sentence_model.pkl','rb'))

# Loading the saved model
model_loaded = BERTopic.load("model_list_versions/topics_7/topics_7_model",embedding_model=sentence_model)

In [3]:
model_loaded.get_topic_info()

,Topic,Count,Name,CustomName
0,0,1774,0_the_co_https co_https,sports
1,1,1009,1_co_https co_https_the,ai and healthcare
2,2,975,2_ai_healthcare_co_https,disaster
3,3,499,3_virus_jdc_the_co,nigeria and politics
4,4,311,4_nigeriadecides2023_gt_your period_period,NaN


In [4]:
model_loaded.get_topic_info(1)

,Topic,Count,Name,CustomName
0,1,1009,1_co_https co_https_the,ai and healthcare


In [5]:
model_loaded.get_topic_info()

,Topic,Count,Name,CustomName
0,0,1774,0_the_co_https co_https,sports
1,1,1009,1_co_https co_https_the,ai and healthcare
2,2,975,2_ai_healthcare_co_https,disaster
3,3,499,3_virus_jdc_the_co,nigeria and politics
4,4,311,4_nigeriadecides2023_gt_your period_period,NaN


### **Suggesting links to after Topic classification**

In [61]:
topic_mapping = {
    0:"usa and politics",
    1:"sports",
    2:'ai and healthcare',
    3:'disaster/disease',
    4:"nigeria and politics"
}

In [69]:
suggest = {
    'usa and politics': "https://www.foxnews.com/us"
    ,'ai and healthcare':'https://www.google.com/search?sxsrf=APwXEdeCxz_laAWroV0WMZEzapCFt9GKEw:1686747497145&q=ai+and+healthcare&tbm=nws&sa=X&ved=2ahUKEwi_kJKL6ML_AhXOUMAKHQCHCgkQ0pQJegQIDhAB&biw=1536&bih=746&dpr=1.25'
    ,'disaster/disease':"https://www.google.com/search?q=disaster+and+disease&biw=1536&bih=746&tbm=nws&sxsrf=APwXEdd0vK4IwALsn0EwzyCcJ8SDrdX00A%3A1686747633393&ei=8bmJZOXDF4bbgQa0z6CQCg&ved=0ahUKEwilgI7M6ML_AhWGbcAKHbQnCKIQ4dUDCA0&uact=5&oq=disaster+and+disease&gs_lcp=Cgxnd3Mtd2l6LW5ld3MQAzIECAAQHjIGCAAQCBAeMgYIABAIEB4yCAgAEIoFEIYDMggIABCKBRCGA1D1Blj1BmDiB2gAcAB4AIAB_wGIAe0DkgEDMi0ymAEAoAEBwAEB&sclient=gws-wiz-news"
    ,"nigeria and politics":"https://www.google.com/search?q=nigeria+and+politics&biw=1536&bih=746&tbm=nws&sxsrf=APwXEddGSpQ3iiJmSPzR5Cfd5VoXTtfGMw%3A1686747637116&ei=9bmJZOzdBpqTgQbB9rTgDw&ved=0ahUKEwisrPHN6ML_AhWaScAKHUE7DfwQ4dUDCA0&uact=5&oq=nigeria+and+politics&gs_lcp=Cgxnd3Mtd2l6LW5ld3MQAzIFCAAQgAQyBQgAEIAEMgYIABAWEB4yBggAEBYQHjIGCAAQFhAeMgYIABAWEB4yBggAEBYQHjIGCAAQFhAeMgYIABAWEB4yBggAEBYQHjoICAAQigUQhgM6CAgAEIoFEJECOggIABCABBCxAzoLCAAQigUQsQMQgwE6CwgAEIAEELEDEIMBOgsIABCKBRCxAxCRAjoKCAAQigUQsQMQQzoNCAAQigUQsQMQgwEQCjoHCAAQigUQQ1DMB1jAKGCnKmgEcAB4AYAB9gKIAZ42kgEGMi02LjE2mAEAoAEBwAEB&sclient=gws-wiz-news"
    ,'sports': {'arsenal':"https://www.newsnow.com/ng/Sport/Football/Premier+League/Arsenal",
                "chelsea":"https://www.newsnow.com/ng/Sport/Football/Premier+League/Chelsea",
                "psg":"https://www.newsnow.com/ng/?search=psg&lang=en",
                " ":"https://www.newsnow.co.uk/h/Sport/Football/Transfer+News",
                "basketball":"https://www.newsnow.com/ng/Sport/NBA"
                    }
}

In [70]:
# testing 
topic_mapping[0]

'usa and politics'

In [86]:
# how to access nested dictionary
suggest['sports']['arsenal']
suggest['sports'].keys()

dict_keys(['arsenal', 'chelsea', 'psg', ' ', 'basketball'])

In [105]:
# https://www.kaggle.com/code/meetnagadia/topic-modeling-using-bertopic/notebook#Testing-the-model
def use_compute(texts):
    # embeddings = sentence_model.encode(text, show_progress_bar=True)
    topics_new, probs_new = model_loaded.transform(texts,)
    topic_mapped = topic_mapping[topics_new[0]]
    for text in texts:
        print(text,"\n")
        print("-----------------------------------------------------------------------------------")
        print(f"Suggested Topic of the texts: {topic_mapped}")
        
        # checking if topic mapping is in the dictionary keys
        if topic_mapped in suggest.keys():
            # ensuring the topic is not sports
            if topic_mapped != 'sports':
                # suggesting the link related to the topic
                print(f"This is the suggested link: {suggest[topic_mapped]}")
            # if the topic is a related to sports
            elif topic_mapped == 'sports':
                # looping over the keys of sports keys
                for word in suggest['sports'].keys():
                    # searching the texts to see if any of the words is related to 
                    # the keys in within sports
                    for text in texts:
                        # lower case the slotted text to enable proper searching of the texts
                        if word in text.lower():
                            print(f"This is the suggested link: {suggest['sports'][word]}")
            

In [106]:
text = ["""The current state of Nigeria is quite dire and we should look forward to proper solution!
What can we as a Niation do? Build from the ground up once again? Should our independence be forfeitted or should chek ourselves and ask what went wrong and look towards the solution;
Manifesto for this plan
- Complete overhaul
- Disolvement of corrupt agencies
- Removel of corrupt Head of states

#MAKENIAIJAGREATAGAIN"""]
use_compute(text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The current state of Nigeria is quite dire and we should look forward to proper solution!
What can we as a Niation do? Build from the ground up once again? Should our independence be forfeitted or should chek ourselves and ask what went wrong and look towards the solution;
Manifesto for this plan
- Complete overhaul
- Disolvement of corrupt agencies
- Removel of corrupt Head of states

#MAKENIAIJAGREATAGAIN 

-----------------------------------------------------------------------------------
Suggested Topic of the texts: nigeria and politics
This is the suggested link: https://www.google.com/search?q=nigeria+and+politics&biw=1536&bih=746&tbm=nws&sxsrf=APwXEddGSpQ3iiJmSPzR5Cfd5VoXTtfGMw%3A1686747637116&ei=9bmJZOzdBpqTgQbB9rTgDw&ved=0ahUKEwisrPHN6ML_AhWaScAKHUE7DfwQ4dUDCA0&uact=5&oq=nigeria+and+politics&gs_lcp=Cgxnd3Mtd2l6LW5ld3MQAzIFCAAQgAQyBQgAEIAEMgYIABAWEB4yBggAEBYQHjIGCAAQFhAeMgYIABAWEB4yBggAEBYQHjIGCAAQFhAeMgYIABAWEB4yBggAEBYQHjoICAAQigUQhgM6CAgAEIoFEJECOggIABCABBCxAzoLCAAQigUQsQMQ

In [107]:
text = ["""daniel and the basketball teams has won the EPL"""]
use_compute(text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

daniel and the basketball teams has won the EPL 

-----------------------------------------------------------------------------------
Suggested Topic of the texts: sports
This is the suggested link: https://www.newsnow.co.uk/h/Sport/Football/Transfer+News
This is the suggested link: https://www.newsnow.com/ng/Sport/NBA


In [108]:
text = ["""The harsh plague is real!"""]
use_compute(text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The harsh plague is real! 

-----------------------------------------------------------------------------------
Suggested Topic of the texts: disaster/disease
This is the suggested link: https://www.google.com/search?q=disaster+and+disease&biw=1536&bih=746&tbm=nws&sxsrf=APwXEdd0vK4IwALsn0EwzyCcJ8SDrdX00A%3A1686747633393&ei=8bmJZOXDF4bbgQa0z6CQCg&ved=0ahUKEwilgI7M6ML_AhWGbcAKHbQnCKIQ4dUDCA0&uact=5&oq=disaster+and+disease&gs_lcp=Cgxnd3Mtd2l6LW5ld3MQAzIECAAQHjIGCAAQCBAeMgYIABAIEB4yCAgAEIoFEIYDMggIABCKBRCGA1D1Blj1BmDiB2gAcAB4AIAB_wGIAe0DkgEDMi0ymAEAoAEBwAEB&sclient=gws-wiz-news


In [109]:
text = ["""joe biden is the president of USA"""]
use_compute(text)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

joe biden is the president of USA 

-----------------------------------------------------------------------------------
Suggested Topic of the texts: usa and politics
This is the suggested link: https://www.foxnews.com/us
